<h1>Creation of phenopackets from tabular data (individuals in columns)</h1>
<p>We will process <a href="https://pubmed.ncbi.nlm.nih.gov/9817917/" target="__blank">Strom, et al. (1998) Diabetes insipidus, diabetes mellitus, optic atrophy and deafness (DIDMOAD) caused by mutations in a novel gene (wolframin) coding for a predicted transmembrane protein</a></p>
<p>pyphetools provides a convenient way of extracting HPO terms from typical tables presented in supplemental material. Typical tables can have the individuals in columns or rows. In this case, we extract data from TABLE.</p>
<p>This note shows how to work through the table and set up the pyphetools encoder. The table was copied from table 1 in the original publication.</p>
<p>Users can work on one column at a time and then generate a collection of <a href="https://pubmed.ncbi.nlm.nih.gov/35705716/" target="__blank">GA4GH phenopackets</a> to represent each patient included in the original supplemental material. These phenopackets can then be used for a variety of downstream applications.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import os
import sys
sys.path.insert(0, os.path.abspath('../../../pyphetools'))
sys.path.insert(0, os.path.abspath('../../pyphetools'))
from pyphetools.creation import *
from pyphetools.creation.simple_column_mapper import try_mapping_columns
import numpy as np

<h2>Importing HPO data</h2>
<p>pyphetools uses the Human Phenotype Ontology (HPO) to encode phenotypic features. The recommended way of doing this is to ingest the hp.json file using HpoParser, which in turn creates an HpoConceptRecognizer object. </p>
<p>The HpoParser can accept a hpo_json_file argument if you want to use a specific file. If the argument is not passed, it will download the latext hp.json file from the HPO GitHub site and store it in a new subdirectory called hpo_data. It will not download the file if the file is already downloaded.</p>

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155")
metadata.default_versions_with_hpo(version=hpo_version)

<h2>Importing the supplemental table</h2>
<p>Here, we use the pandas library to import this file (note that the Python package called openpyxl must be installed to read Excel files with pandas, although the library does not need to be imported in this notebook). pyphetools expects a pandas DataFrame as input, and users can choose any input format available for pandas include CSV, TSV, and Excel, or can use any other method to transform their input data into a Pandas DataFrame before using pyphetools.</p>

In [3]:
df = pd.read_excel('../../data/WFS1/PMID_9817917.xlsx')

In [4]:
df

,Family,Patient,Sex,Age,Diabetes mellitus,Progressive optic atrophy,Hearing impairment,Diabetes insipidus,Abnormality of the kidney,Neurological abnormalities,Other complications,Consangui,Variant
0,1.0,5519,f,22.0,+,+,+,+,+,"Ataxia, nystagmus","Retarded sexual maturation, depression",-,1380del9
1,1.0,13883,f,11.0,+,-,-,-,+,-,-,-,1380del9
2,2.0,13775,f,20.0,+,+,+,+,-,-,-,-,460+1G>A
3,2.0,13776,m,17.0,+,+,+,+,+,-,Retarded sexual maturation,-,460+1G>A
4,4.0,13070,f,22.0,+,+,+,-,+,Abnormal EEG,Psychiatric illness,-,599delT
5,5.0,13885,f,35.0,+,+,+,-,+,-,Cataract,-,1096C>T
6,6.0,13062,f,25.0,+,+,+,+,+,"Ataxia, nystagmus",NaN,NaN,676C>T
7,7.0,13076,m,26.0,+,+,+,+,-,-,"Retarded sexual maturation, mental retardation",-,599delT
8,8.0,13073,f,35.0,+,+,+,-,+,Ataxia,"Cataract, psychiatric illness, ragged red fibers",NaN,1096C>T
9,9.0,13781,m,19.0,+,+,+,+,+,Abnormal EEG,Retarded sexual maturation,+,1558C>T


Some column names might include spaces in front or after, and a couple of columns are subheadings and only contain NaNs, so lets correct that. Furthermore, remove individuals without an age specified or a variant in this gene.

In [5]:
df.columns = df.columns.str.strip()
df = df.dropna(axis=1, how='all')
df['patient_id'] = df['Patient']
df = df[~df['Age'].isna()]
df = df[~df['Variant'].isna()]
df

,Family,Patient,Sex,Age,Diabetes mellitus,Progressive optic atrophy,Hearing impairment,Diabetes insipidus,Abnormality of the kidney,Neurological abnormalities,Other complications,Consangui,Variant,patient_id
0,1.0,5519,f,22.0,+,+,+,+,+,"Ataxia, nystagmus","Retarded sexual maturation, depression",-,1380del9,5519
1,1.0,13883,f,11.0,+,-,-,-,+,-,-,-,1380del9,13883
2,2.0,13775,f,20.0,+,+,+,+,-,-,-,-,460+1G>A,13775
3,2.0,13776,m,17.0,+,+,+,+,+,-,Retarded sexual maturation,-,460+1G>A,13776
4,4.0,13070,f,22.0,+,+,+,-,+,Abnormal EEG,Psychiatric illness,-,599delT,13070
5,5.0,13885,f,35.0,+,+,+,-,+,-,Cataract,-,1096C>T,13885
6,6.0,13062,f,25.0,+,+,+,+,+,"Ataxia, nystagmus",NaN,NaN,676C>T,13062
7,7.0,13076,m,26.0,+,+,+,+,-,-,"Retarded sexual maturation, mental retardation",-,599delT,13076
8,8.0,13073,f,35.0,+,+,+,-,+,Ataxia,"Cataract, psychiatric illness, ragged red fibers",NaN,1096C>T,13073
9,9.0,13781,m,19.0,+,+,+,+,+,Abnormal EEG,Retarded sexual maturation,+,1558C>T,13781


<h2>Column mappers</h2>
<p>Please see the notebook "Create phenopackets from tabular data with individuals in rows" for explanations. In the following cell we create a dictionary for the ColumnMappers. Note that the code is identical except that we use the df.loc function to get the corresponding row data</p>

In [6]:
hpo_cr = parser.get_hpo_concept_recognizer()
column_mapper_d, col_not_found = try_mapping_columns(df=df,
                                                    observed='+',
                                                    excluded='-',
                                                    hpo_cr=hpo_cr,
                                                    preview=True)

                             term    status
0  Diabetes mellitus (HP:0000819)  observed
1  Diabetes mellitus (HP:0000819)  observed
2  Diabetes mellitus (HP:0000819)  observed
3  Diabetes mellitus (HP:0000819)  observed
4  Diabetes mellitus (HP:0000819)  observed
5  Diabetes mellitus (HP:0000819)  observed
6  Diabetes mellitus (HP:0000819)  observed
7  Diabetes mellitus (HP:0000819)  observed
8  Diabetes mellitus (HP:0000819)  observed
9  Diabetes mellitus (HP:0000819)  observed
                         term    status
0  Optic atrophy (HP:0000648)  observed
1  Optic atrophy (HP:0000648)  excluded
2  Optic atrophy (HP:0000648)  observed
3  Optic atrophy (HP:0000648)  observed
4  Optic atrophy (HP:0000648)  observed
5  Optic atrophy (HP:0000648)  observed
6  Optic atrophy (HP:0000648)  observed
7  Optic atrophy (HP:0000648)  observed
8  Optic atrophy (HP:0000648)  observed
9  Optic atrophy (HP:0000648)  observed
                              term    status
0  Hearing impairment (HP:00003

In [7]:
print(col_not_found)

['Family', 'Patient', 'Sex', 'Age', 'Neurological abnormalities', 'Other complications', 'Consangui', 'Variant', 'patient_id']


In [8]:
neurological = {'Ataxia': 'Ataxia',
                 'nystagmus': 'Nystagmus',
               'Abnormal EEG': 'EEG abnormality'}
neurologicalMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=neurological)
print(neurologicalMapper.preview_column(df['Neurological abnormalities']))
column_mapper_d['Neurological abnormalities'] = neurologicalMapper

other = {'Retarded sexual maturation': 'Puberty and gonadal disorders',
                 'depression': 'Depression',
               'psychiatric illness': 'Behavioral abnormality',
                'cataract' : 'Cataract', 
         'mental retardation': 'Intellectual disability',
         'ragged red fibers': 'Ragged-red muscle fibers'
        }
otherMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=other)
print(otherMapper.preview_column(df['Other complications']))
column_mapper_d['Other complications'] = otherMapper

                                                           terms
0  HP:0001251 (Ataxia/observed); HP:0000639 (Nystagmus/observed)
1                                                            n/a
2                                                            n/a
3                                                            n/a
4                          HP:0002353 (EEG abnormality/observed)
5                                                            n/a
6  HP:0001251 (Ataxia/observed); HP:0000639 (Nystagmus/observed)
7                                                            n/a
8                                   HP:0001251 (Ataxia/observed)
9                          HP:0002353 (EEG abnormality/observed)
                                                                                                                          terms
0                                         HP:0008373 (Puberty and gonadal disorders/observed); HP:0000716 (Depression/observed)
1                            

<h2>Variant Data</h2>
<p>The variant data (HGVS< transcript) is listed in the Variant (hg19, NM_015133.4) column.</p>

In [9]:
genome = 'hg38'
default_genotype = 'heterozygous'
transcript='NM_006005.3'
varMapper = VariantColumnMapper(assembly=genome,column_name='Variant', 
                                transcript=transcript, genotype=default_genotype)

<h1>Demographic data</h1>
<p>pyphetools can be used to capture information about age, sex, and individual identifiers. This information is stored in a map of "IndividualMapper" objects. Special treatment may be required for the indifiers, which may be used as the column names or row index.</p>

In [10]:
ageMapper = AgeColumnMapper.by_year('Age')
ageMapper.preview_column(df['Age'])

,original column contents,age
0,22.0,P22Y0M
1,11.0,P11Y0M
2,20.0,P20Y0M
3,17.0,P17Y0M
4,35.0,P35Y0M
5,25.0,P25Y0M
6,26.0,P26Y0M
7,19.0,P19Y0M


In [11]:
sexMapper = SexColumnMapper(male_symbol='m', female_symbol='f', column_name='Sex')
sexMapper.preview_column(df['Sex'])

,original column contents,sex
0,f,FEMALE
1,f,FEMALE
2,f,FEMALE
3,m,MALE
4,f,FEMALE
5,f,FEMALE
6,f,FEMALE
7,m,MALE
8,f,FEMALE
9,m,MALE


In [12]:
pmid = "PMID: 9817917"
encoder = CohortEncoder(df=df, hpo_cr=hpo_cr, column_mapper_d=column_mapper_d, 
                        individual_column_name="patient_id", agemapper=ageMapper, sexmapper=sexMapper,
                       variant_mapper=varMapper, metadata=metadata,
                       pmid=pmid)
encoder.set_disease(disease_id='222300', label='WOLFRAM SYNDROME 1')

In [13]:
individuals = encoder.get_individuals()

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A1380del9/NM_006005.3
Not able to get variant for 1380del9: Expecting to get a gene_variant from Variant Validator but got warning
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A1380del9/NM_006005.3
Not able to get variant for 1380del9: Expecting to get a gene_variant from Variant Validator but got warning
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A460+1G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A460+1G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A599delT/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A1096C>T/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A676C>T/NM_006005.3
https://rest.variantvalidator.

In [14]:
i1 = individuals[0]
phenopacket1 = i1.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh())
json_string = MessageToJson(phenopacket1)
print(json_string)

{
  "id": "5519",
  "subject": {
    "id": "5519",
    "timeAtLastEncounter": {
      "age": {
        "iso8601duration": "P22Y0M"
      }
    },
    "sex": "FEMALE"
  },
  "phenotypicFeatures": [
    {
      "type": {
        "id": "HP:0000819",
        "label": "Diabetes mellitus"
      },
      "onset": {
        "age": {
          "iso8601duration": "P22Y0M"
        }
      }
    },
    {
      "type": {
        "id": "HP:0000648",
        "label": "Optic atrophy"
      },
      "onset": {
        "age": {
          "iso8601duration": "P22Y0M"
        }
      }
    },
    {
      "type": {
        "id": "HP:0000365",
        "label": "Hearing impairment"
      },
      "onset": {
        "age": {
          "iso8601duration": "P22Y0M"
        }
      }
    },
    {
      "type": {
        "id": "HP:0000873",
        "label": "Diabetes insipidus"
      },
      "onset": {
        "age": {
          "iso8601duration": "P22Y0M"
        }
      }
    },
    {
      "type": {
        "id

In [15]:
output_directory = "../../phenopackets/WFS1/"
encoder.output_phenopackets(outdir=output_directory)

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A1380del9/NM_006005.3
Not able to get variant for 1380del9: Expecting to get a gene_variant from Variant Validator but got warning
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A1380del9/NM_006005.3
Not able to get variant for 1380del9: Expecting to get a gene_variant from Variant Validator but got warning
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A460+1G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A460+1G>A/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A599delT/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A1096C>T/NM_006005.3
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_006005.3%3A676C>T/NM_006005.3
https://rest.variantvalidator.